# Caso práctico de Analítica Escalable (Ejercicios) #

En este notebook, se van a realizar los ejercicios del módulo. En lugar de tener contenido teórico y descripciones, se dejarán únicamente las celdas de código necesarias para su ejecución.

Para completar los ejercicios, hay que codificar y ejecutar la solución en las celdas que se encuentran justo debajo de los enunciados de los ejercicios.

Una vez se haya terminado, en el menú de la izquiera, a la hora de seleccionar el notebook, si se le hace click a la flecha que se encuentra en la derecha, se puede exportar al notebook. Hay que exportarlo en formato DBC (Databricks Notebook) como en HTML.

In [3]:
print sc.version

File "<command-4027673054087824>" , line 1 
 print sc.version 
 ^ 
 SyntaxError : Missing parentheses in call to 'print'

Los ejercicios consistirán en añadir nuevas funcionalidades, o ejecutar nuevo código, sobre el Notebook que contiene toda la teoría vista en el módulo. Por ello, gran parte del código que se encuentra dentro del notebook de contenido teórico se encontrará aquí de nuevo, pero se pedirá nuevo código.

## Importando los datos ##

In [6]:
dbutils.fs.cp("/FileStore/tables/Hotel_Reviews.csv", "file:///databricks/driver/Hotel_Reviews.csv")

Out[ 2 ]: True

In [7]:
def score_to_string(score):
  if score < 5:
    return "Bad"
  elif score < 7:
    return "Normal"
  elif score < 9:
    return "Good"
  elif score < 10: 
    return "Excellent"
  else:
    return "Perfect"
  
def score_to_evaluation(score_string):
  score_dict = {
    "Bad": 0,
    "Normal": 1,
    "Good": 2,
    "Excellent": 3,
    "Perfect": 4
  }
  return score_dict.get(score_string, None)

## DataFrames en Spark: SparkSQL. ##

In [9]:
df_spark_sql = spark.read.format("csv")\
         .option("header", "true")\
         .option("inferSchema", "true")\
         .load("file:///databricks/driver/Hotel_Reviews.csv")

In [10]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType

score_string_udf = udf(score_to_string, StringType())
score_evaluation_udf = udf(score_to_evaluation, IntegerType())

In [11]:
df_spark_sql = df_spark_sql.withColumn('score_string',score_string_udf(df_spark_sql["Average_Score"]))
df_spark_sql = df_spark_sql.withColumn('score_evaluation',score_evaluation_udf(df_spark_sql["score_string"]))

In [12]:
def day_to_int(day):
  return int(day.replace(" days", "").replace(" day", ""))
day_to_int_udf = udf(day_to_int, IntegerType())
df_spark_sql = df_spark_sql.withColumn("days_since_review", day_to_int_udf(df_spark_sql["days_since_review"]))

### Ejercicio 1: Crear un bucle que muestre todas las columnas del DataFrame, junto con sus tipos. También puedes pintar el esquema del Dataframe. ###

In [14]:
# Escribir aquí el ejericicio
# 
print("Primero imprimimos el esquema:")
print(df_spark_sql.printSchema())
print("Ahora hacemos el bucle que muestra las columnas y tipos del dataFrame")
for name, dtype in df_spark_sql.dtypes:
  print(name, " --> ",dtype)

Primero imprimimos el esquema:
root
-- Hotel_Address: string (nullable = true)
-- Additional_Number_of_Scoring: integer (nullable = true)
-- Review_Date: string (nullable = true)
-- Average_Score: double (nullable = true)
-- Hotel_Name: string (nullable = true)
-- Reviewer_Nationality: string (nullable = true)
-- Negative_Review: string (nullable = true)
-- Review_Total_Negative_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews: integer (nullable = true)
-- Positive_Review: string (nullable = true)
-- Review_Total_Positive_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
-- Reviewer_Score: double (nullable = true)
-- Tags: string (nullable = true)
-- days_since_review: integer (nullable = true)
-- lat: string (nullable = true)
-- lng: string (nullable = true)
-- score_string: string (nullable = true)
-- score_evaluation: integer (nullable = true)

None
Ahora hacemos el bucle que muestra las columnas y tipos del dataFrame
Hotel_Address --> string
Additional_Number_of_Scoring --> int
Review_Date --> string
Average_Score --> double
Hotel_Name --> string
Reviewer_Nationality --> string
Negative_Review --> string
Review_Total_Negative_Word_Counts --> int
Total_Number_of_Reviews --> int
Positive_Review --> string
Review_Total_Positive_Word_Counts --> int
Total_Number_of_Reviews_Reviewer_Has_Given --> int
Reviewer_Score --> double
Tags --> string
days_since_review --> int
lat --> string
lng --> string
score_string --> string
score_evaluation --> int

### Ejercicio 2: Realizar un muestreo de 10 valores únicos de nombres de hoteles. Ordénalos alfanuméricamente de forma ascendente (primero los números 0-9, después A-Z). ###

In [16]:
# Escribir aquí el ejercicio
df_spark_sql.select('Hotel_Name').distinct().limit(10).orderBy("Hotel_Name").show()

+--------------------+
 Hotel_Name|
+--------------------+
 Avenida Palace|
Best Western Sera...|
Grange Holborn Hotel|
H tel Barri re Le...|
H tel Elysees Mermoz|
 H10 Port Vell 4 Sup|
 HCC Regente|
Hyatt Regency Ams...|
Melia Paris Notre...|
Melia Paris Tour ...|
+--------------------+

### Ejercicio 3: Transforma las columnas *lat* y *lng* al tipo Float.

In [18]:
# Transformación de la columna sin utilizar UDF.
from pyspark.sql.types import FloatType

df_new_aux = df_spark_sql.withColumn("lat", df_spark_sql["lat"].cast(FloatType()) )
df_new_sql = df_new_aux.withColumn("lng", df_spark_sql["lng"].cast(FloatType()) )
df_spark_sql = df_new_sql
print ('  ----  Comprobamos que ha cambiado ------ ')
print(df_spark_sql.printSchema())

---- Comprobamos que ha cambiado ------ 
root
-- Hotel_Address: string (nullable = true)
-- Additional_Number_of_Scoring: integer (nullable = true)
-- Review_Date: string (nullable = true)
-- Average_Score: double (nullable = true)
-- Hotel_Name: string (nullable = true)
-- Reviewer_Nationality: string (nullable = true)
-- Negative_Review: string (nullable = true)
-- Review_Total_Negative_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews: integer (nullable = true)
-- Positive_Review: string (nullable = true)
-- Review_Total_Positive_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
-- Reviewer_Score: double (nullable = true)
-- Tags: string (nullable = true)
-- days_since_review: integer (nullable = true)
-- lat: float (nullable = true)
-- lng: float (nullable = true)
-- score_string: string (nullable = true)
-- score_evaluation: integer (nullable = true)

None

In [19]:
splits = df_spark_sql.randomSplit([0.67, 0.33])
df_spark_sql_train = splits[0].dropna()
df_spark_sql_test = splits[1].dropna()
print(df_spark_sql_train.count())
print(df_spark_sql_test.count())

343059
169411

### Ejercicio 4: ¿Cuántos hoteles tienen una puntuación de 'Perfect'? ¿Y 'Good'? ¿Y 'Normal' junto a 'Good'? (Utilizar el dataset de Train)

In [21]:
# Escribir aquí el ejercicio
print ('  ----  Cantidad de Perfect ------ ')
df_spark_sql_train.select('score_string').filter("score_string = 'Perfect'").groupBy('score_string').count().show()
print ('  ----  Cantidad de Good ------ ')
df_spark_sql_train.select('score_string').filter("score_string = 'Good'").groupBy('score_string').count().show()
print ('  ----  Cantidad de Normal y Good ------ ')
df_spark_sql_train.select('score_string').filter("score_string = 'Normal' or score_string = 'Good'").groupBy('score_string').count().show()

---- Cantidad de Perfect ------ 
+------------+-----+
score_string|count|
+------------+-----+
+------------+-----+

 ---- Cantidad de Good ------ 
+------------+------+
score_string| count|
+------------+------+
 Good|286020|
+------------+------+

 ---- Cantidad de Normal y Good ------ 
+------------+------+
score_string| count|
+------------+------+
 Good|286020|
 Normal| 3769|
+------------+------+

### Ejercicio 5: Obtener los hoteles con mayor puntuación media, descartando todos los que tengan una puntuación por encima de Good. (Utilizar el dataset de Train) ###

In [23]:
# Escribir aquí el ejercicio
from pyspark.sql.functions import desc
# df_spark_sql_train.limit(1000).show()
print ('  ----  10 Hoteles no Excelent con mayor puntuacion media ------ ')
df_Puntuacion = df_spark_sql_train.filter("score_string <> 'Excellent'").groupBy('Hotel_Name').avg('Reviewer_Score')
df_Puntuacion.orderBy(desc("avg(Reviewer_Score)")).limit(10).show()

---- 10 Hoteles no Excelent con mayor puntuacion media ------ 
+--------------------+-------------------+
 Hotel_Name|avg(Reviewer_Score)|
+--------------------+-------------------+
Hotel Stendhal Pl...| 9.595454545454546|
 XO Hotel| 9.450000000000001|
 Drawing Hotel| 9.445454545454545|
H tel Saint Paul ...| 9.321875000000002|
 Hotel Daniel Paris| 9.2875|
 Splendid Etoile| 9.284931506849318|
Hotel Des Saints ...| 9.269117647058826|
H tel Bourgogne M...| 9.243373493975904|
Hotel Beethoven Wien| 9.233035714285714|
 H tel Le Walt| 9.227906976744187|
+--------------------+-------------------+

# Machine Learning en Apache Spark: Spark MLLib y Spark ML #

## Clasificación Supervisada: Árboles de decisión ##

### Ejercicio 6.1: Volver a observar todas las columnas del dataframe, para identificar las que sean categóricas. ###

In [27]:
# Escribir aquí el ejercicio
print("   ---------- Primero imprimimos el esquema: -----------")
print(df_spark_sql.printSchema())
print("   ---------- Despues imprimimos algunos datos: -----------")
df_spark_sql.select('*').limit(100).show()

---------- Primero imprimimos el esquema: -----------
root
-- Hotel_Address: string (nullable = true)
-- Additional_Number_of_Scoring: integer (nullable = true)
-- Review_Date: string (nullable = true)
-- Average_Score: double (nullable = true)
-- Hotel_Name: string (nullable = true)
-- Reviewer_Nationality: string (nullable = true)
-- Negative_Review: string (nullable = true)
-- Review_Total_Negative_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews: integer (nullable = true)
-- Positive_Review: string (nullable = true)
-- Review_Total_Positive_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
-- Reviewer_Score: double (nullable = true)
-- Tags: string (nullable = true)
-- days_since_review: integer (nullable = true)
-- lat: float (nullable = true)
-- lng: float (nullable = true)
-- score_string: string (nullable = true)
-- score_evaluation: integer (nullable = true)

None
 ---------- Despues imprimimos algunos datos: -----------
+--------------------+----------------------------+-----------+-------------+-----------+--------------------+--------------------+---------------------------------+-----------------------+--------------------+---------------------------------+------------------------------------------+--------------+--------------------+-----------------+---------+---------+------------+----------------+
 Hotel_Address|Additional_Number_of_Scoring|Review_Date|Average_Score| Hotel_Name|Reviewer_Nationality| Negative_Review|Review_Total_Negative_Word_Counts|Total_Number_of_Reviews| Positive_Review|Review_Total_Positive_Word_Counts|Total_Number_of_Reviews_Reviewer_Has_Given|Reviewer_Score| Tags|days_since_review| lat| lng|score_string|score_evaluation|
+--------------------+----------------------------+-----------+-------------+-----------+--------------------+--------------------+---------------------------------+-----------------------+--------------------+---------------------------------+------------------------------------------+--------------+--------------------+-----------------+---------+---------+------------+----------------+
 s Gravesandestra...| 194| 8/3/2017| 7.7|Hotel Arena| Russia | I am so angry th...| 397| 1403| Only the park ou...| 11| 7| 2.9|[' Leisure trip '...| 0|52.360577|4.9159684| Good| 2|
 s Gravesandestra...| 194| 8/3/2017| 7.7|Hotel Arena| Ireland | No Negative| 0| 1403| No real complain...| 105| 7| 7.5|[' Leisure trip '...| 0|52.360577|4.9159684| Good| 2|
 s Gravesandestra...| 194| 7/31/2017| 7.7|Hotel Arena| Australia | Rooms are nice b...| 42| 1403| Location was goo...| 21| 9| 7.1|[' Leisure trip '...| 3|52.360577|4.9159684| Good| 2|
 s Gravesandestra...| 194| 7/31/2017| 7.7|Hotel Arena| United Kingdom | My room was dirt...| 210| 1403| Great location i...| 26| 1| 3.8|[' Leisure trip '...| 3|52.360577|4.9159684| Good| 2|
 s Gravesandestra...| 194| 7/24/2017| 7.7|Hotel Arena| New Zealand | You When I booke...| 140| 1403| Amazing location...| 8| 3| 6.7|[' Leisure trip '...| 10|52.360577|4.9159684| Good| 2|
 s Gravesandestra...| 194| 7/24/2017| 7.7|Hotel Arena| Poland | Backyard of the ...| 17| 1403| Good restaurant ...| 20| 1| 6.7|[' Leisure trip '...| 10|52.360577|4.9159684| Good| 2|
 s Gravesandestra...| 194| 7/17/2017| 7.7|Hotel Arena| United Kingdom | Cleaner did not ...| 33| 1403| The room is spac...| 18| 6| 4.6|[' Leisure trip '...| 17|52.360577|4.9159684| Good| 2|
 s Gravesandestra...| 194| 7/17/2017| 7.7|Hotel Arena| United Kingdom | Apart from the p...| 11| 1403| Good location Se...| 19| 1| 10.0|[' Leisure trip '...| 17|52.360577|4.9159684| Good| 2|
 s Gravesandestra...| 194| 7/9/2017| 7.7|Hotel Arena| Belgium | Even though the ...| 34| 1403| No Positive| 0| 3| 6.5|[' Leisure trip '...| 25|52.360577|4.9159684| Good| 2|
 s Gravesandestra...| 194| 7/8/2017| 7.7|Hotel Arena| Norway | The aircondition...| 15| 1403| The room was big...| 50| 1| 7.9|[' Leisure trip '...| 26|52.360577|4.9159684| Good| 2|
 s Graves

### Ejercicio 6.2: Eliminar, de los dataframes df_spark_sql_train y df_spark_sql test, las variables 'Hotel_Address', 'Hotel_Name', 'Tags', 'Positive Review', 'Negative_Review' y 'score_string'. Llamarlos: df_DT_train y df_DT_test. ###

In [29]:
# Escribir aquí el ejercicio
df_DT_train = df_spark_sql_train.drop('Hotel_Address', 'Hotel_Name', 'Tags', 'Positive_Review', 'Negative_Review', 'score_string')
print("   ---------- Comprobamos que se han eliminado las columnas de df_DT_train -----------")
print(df_DT_train.printSchema())
df_DT_test = df_spark_sql_test.drop('Hotel_Address', 'Hotel_Name', 'Tags', 'Positive_Review', 'Negative_Review', 'score_string')
print("   ---------- Comprobamos que se han eliminado las columnas de df_DT_test -----------")
print(df_DT_test.printSchema())

---------- Comprobamos que se han eliminado las columnas de df_DT_train -----------
root
-- Additional_Number_of_Scoring: integer (nullable = true)
-- Review_Date: string (nullable = true)
-- Average_Score: double (nullable = true)
-- Reviewer_Nationality: string (nullable = true)
-- Review_Total_Negative_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews: integer (nullable = true)
-- Review_Total_Positive_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
-- Reviewer_Score: double (nullable = true)
-- days_since_review: integer (nullable = true)
-- lat: float (nullable = true)
-- lng: float (nullable = true)
-- score_evaluation: integer (nullable = true)

None
 ---------- Comprobamos que se han eliminado las columnas de df_DT_test -----------
root
-- Additional_Number_of_Scoring: integer (nullable = true)
-- Review_Date: string (nullable = true)
-- Average_Score: double (nullable = true)
-- Reviewer_Nationality: string (nullable = true)
-- Review_Total_Negative_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews: integer (nullable = true)
-- Review_Total_Positive_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
-- Reviewer_Score: double (nullable = true)
-- days_since_review: integer (nullable = true)
-- lat: float (nullable = true)
-- lng: float (nullable = true)
-- score_evaluation: integer (nullable = true)

None

### Ejercicio 7: Para cada columa restante que sea String ('Review_Date' y 'Review_Nationality'), aplicar un StringIndexer(), devolviendo como resultado la misma columna, pero con su nombre acabando en _index. Sobreescribir ambos dataframes.  ###

In [31]:
# Escribir aquí el ejercicio
from pyspark.ml.feature import StringIndexer

print("   ---------- Definimos un indexer para cada columna -----------")
indexerDate = StringIndexer(inputCol="Review_Date", outputCol="Review_Date_index")
indexerNationality = StringIndexer(inputCol="Reviewer_Nationality", outputCol="Reviewer_Nationality_index")

print("   ---------- Ejecutamos ambos indexers sobre ambos dataframes -----------")
df_DT_train = indexerDate.fit(df_DT_train).transform(df_DT_train)
df_DT_train = indexerNationality.fit(df_DT_train).transform(df_DT_train)
df_DT_test = indexerDate.fit(df_DT_test).transform(df_DT_test)
df_DT_test = indexerNationality.fit(df_DT_test).transform(df_DT_test)
print("   ---------- Comprobamos que se han creado las columnas de df_DT_train -----------")
df_DT_train.select('Review_Date','Review_Date_Index','Reviewer_Nationality','Reviewer_Nationality_Index').limit(100).show()
print("   ---------- Comprobamos que se han creado las columnas de df_DT_test -----------")
df_DT_test.select('Review_Date','Review_Date_Index','Reviewer_Nationality','Reviewer_Nationality_Index').limit(100).show()


---------- Definimos un indexer para cada columna -----------
 ---------- Ejecutamos ambos indexers sobre ambos dataframes -----------
 ---------- Comprobamos que se han creado las columnas de df_DT_train -----------
+-----------+-----------------+--------------------+--------------------------+
Review_Date|Review_Date_Index|Reviewer_Nationality|Reviewer_Nationality_Index|
+-----------+-----------------+--------------------+--------------------------+
 1/12/2016| 115.0| United Kingdom | 0.0|
 1/12/2016| 115.0| United Kingdom | 0.0|
 1/15/2016| 708.0| Australia | 2.0|
 1/15/2016| 708.0| United Kingdom | 0.0|
 1/19/2016| 44.0| Ireland | 3.0|
 1/19/2016| 44.0| United Kingdom | 0.0|
 1/19/2016| 44.0| United Kingdom | 0.0|
 1/19/2017| 515.0| United Kingdom | 0.0|
 1/2/2016| 390.0| United Kingdom | 0.0|
 1/23/2016| 670.0| Ireland | 3.0|
 1/24/2017| 212.0| United Kingdom | 0.0|
 1/26/2016| 21.0| United Kingdom | 0.0|
 1/26/2016| 21.0| United Kingdom | 0.0|
 1/27/2017| 509.0| United Kingdom | 0.0|
 1/3/2016| 14.0| Israel | 11.0|
 1/3/2016| 14.0| United Kingdom | 0.0|
 1/3/2016| 14.0| United Kingdom | 0.0|
 1/3/2016| 14.0| United Kingdom | 0.0|
 1/3/2017| 89.0| Ireland | 3.0|
 1/5/2016| 96.0| Malta | 40.0|
+-----------+-----------------+--------------------+--------------------------+
only showing top 20 rows

 ---------- Comprobamos que se han creado las columnas de df_DT_test -----------
+-----------+-----------------+--------------------+--------------------------+
Review_Date|Review_Date_Index|Reviewer_Nationality|Reviewer_Nationality_Index|
+-----------+-----------------+--------------------+--------------------------+
 1/11/2016| 571.0| Brazil | 35.0|
 1/12/2016| 123.0| United Kingdom | 0.0|
 1/16/2016| 545.0| Turkey | 14.0|
 1/21/2017| 609.0| Ireland | 3.0|
 1/22/2016| 721.0| United Kingdom | 0.0|
 1/22/2017| 362.0| Switzerland | 7.0|
 1/25/2017| 432.0| South Africa | 18.0|
 1/29/2017| 250.0| Denmark | 38.0|
 1/30/2017| 42.0| United Kingdom | 0.0|
 1/4/2017| 220.0| United Kingdom | 0.0|
 1/8/2016| 699.0| United Kingdom | 0.0|
 10/1/2015| 251.0| Australia | 2.0|
 10/11/2016| 426.0| Finland | 39.0|
 10/13/2015| 68.0| Croatia | 48.0|
 10/14/2015| 618.0| Israel | 11.0|
 10/18/2016| 177.0| United Kingdom | 0.0|
 10/26/2016| 321.0| United Kingdom | 0.0|
 10/29/2015| 163.0| United Kingdom | 0.0|
 10/3/2016| 100.0| Slovenia | 60.0|
 10/31/2016| 63.0| Switzerland | 7.0|
+-----------+-----------------+--------------------+--------------------------+
only showing top 20 rows

### Ejercicio 8: Aplicar VectorAssembler() sobre las columnas que no son ni las dos anteriores, ni la columna 'score_evaluation', devolviendo una columna llamada 'features'. Llamar al resultado DT_vector_assembler. ###

In [33]:
# Escribir aquí el ejercicio
from pyspark.ml.feature import VectorAssembler

print("   ---------- Imprimimos el esquema para ver cuales son las columnas a utilizar -----------")
print(df_DT_train.printSchema())

DT_vector_assembler = VectorAssembler(
    inputCols=["Additional_Number_of_Scoring", "Average_Score", "Review_Total_Negative_Word_Counts", "Total_Number_of_Reviews", "Review_Total_Positive_Word_Counts", "Total_Number_of_Reviews_Reviewer_Has_Given", "Reviewer_Score", "days_since_review", "Review_Date_index", "Reviewer_Nationality_index"],
    outputCol="features")

---------- Imprimimos el esquema para ver cuales son las columnas a utilizar -----------
root
-- Additional_Number_of_Scoring: integer (nullable = true)
-- Review_Date: string (nullable = true)
-- Average_Score: double (nullable = true)
-- Reviewer_Nationality: string (nullable = true)
-- Review_Total_Negative_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews: integer (nullable = true)
-- Review_Total_Positive_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
-- Reviewer_Score: double (nullable = true)
-- days_since_review: integer (nullable = true)
-- lat: float (nullable = true)
-- lng: float (nullable = true)
-- score_evaluation: integer (nullable = true)
-- Review_Date_index: double (nullable = false)
-- Reviewer_Nationality_index: double (nullable = false)

None

### Ejercicio 9: Aplicar el transformador sobre ambos dataframes. ###

In [35]:
# Escribir aquí el ejercicio
df_DT_train_assembled = DT_vector_assembler.transform(df_DT_train)
df_DT_test_assembled = DT_vector_assembler.transform(df_DT_test)

print("   ---------- Vemos el dataframe obtenido -----------")
df_DT_train_assembled.select("*").show(10)

---------- Vemos el dataframe obtenido -----------
+----------------------------+-----------+-------------+--------------------+---------------------------------+-----------------------+---------------------------------+------------------------------------------+--------------+-----------------+---------+---------+----------------+-----------------+--------------------------+--------------------+
Additional_Number_of_Scoring|Review_Date|Average_Score|Reviewer_Nationality|Review_Total_Negative_Word_Counts|Total_Number_of_Reviews|Review_Total_Positive_Word_Counts|Total_Number_of_Reviews_Reviewer_Has_Given|Reviewer_Score|days_since_review| lat| lng|score_evaluation|Review_Date_index|Reviewer_Nationality_index| features|
+----------------------------+-----------+-------------+--------------------+---------------------------------+-----------------------+---------------------------------+------------------------------------------+--------------+-----------------+---------+---------+----------------+-----------------+--------------------------+--------------------+
 194| 1/12/2016| 7.7| United Kingdom | 0| 1403| 21| 3| 10.0| 569|52.360577|4.9159684| 2| 115.0| 0.0|[194.0,7.7,0.0,14...|
 194| 1/12/2016| 7.7| United Kingdom | 0| 1403| 13| 1| 9.6| 569|52.360577|4.9159684| 2| 115.0| 0.0|[194.0,7.7,0.0,14...|
 194| 1/15/2016| 7.7| Australia | 6| 1403| 6| 1| 10.0| 566|52.360577|4.9159684| 2| 708.0| 2.0|[194.0,7.7,6.0,14...|
 194| 1/15/2016| 7.7| United Kingdom | 60| 1403| 15| 7| 7.9| 566|52.360577|4.9159684| 2| 708.0| 0.0|[194.0,7.7,60.0,1...|
 194| 1/19/2016| 7.7| Ireland | 9| 1403| 0| 1| 5.4| 562|52.360577|4.9159684| 2| 44.0| 3.0|[194.0,7.7,9.0,14...|
 194| 1/19/2016| 7.7| United Kingdom | 39| 1403| 11| 1| 7.9| 562|52.360577|4.9159684| 2| 44.0| 0.0|[194.0,7.7,39.0,1...|
 194| 1/19/2016| 7.7| United Kingdom | 9| 1403| 8| 1| 8.8| 562|52.360577|4.9159684| 2| 44.0| 0.0|[194.0,7.7,9.0,14...|
 194| 1/19/2017| 7.7| United Kingdom | 15| 1403| 24| 1| 9.2| 196|52.360577|4.9159684| 2| 515.0| 0.0|[194.0,7.7,15.0,1...|
 194| 1/2/2016| 7.7| United Kingdom | 63| 1403| 10| 3| 6.7| 579|52.360577|4.9159684| 2| 390.0| 0.0|[194.0,7.7,63.0,1...|
 194| 1/23/2016| 7.7| Ireland | 0| 1403| 78| 1| 10.0| 558|52.360577|4.9159684| 2| 670.0| 3.0|[194.0,7.7,0.0,14...|
+----------------------------+-----------+-------------+--------------------+---------------------------------+-----------------------+---------------------------------+------------------------------------------+--------------+-----------------+---------+---------+----------------+-----------------+--------------------------+--------------------+
only showing top 10 rows

### Ejercicio 10: Inicializar el modelo de árbol de decisión, entrenarlo y aplicarlo sobre los datos de test. ###
* Modelo: DecisionTreeClassifier:
  * Label: score_evaluation.
  * Features: features.
  * maxBins: 1000
  * maxDepth: 1

In [37]:
# Escribir aquí el ejercicio
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline

print("   ---------- Creamos el arbol de decision para la columna score_evaluation y entrenamos el modelo con los datos TRAIN -----------")
dt = DecisionTreeClassifier(labelCol="score_evaluation", featuresCol="features", maxBins=1000, maxDepth=1)
pipeline = Pipeline(stages=[dt])
model = pipeline.fit(df_DT_train_assembled)
print("   ---------- Aplicamos el modelo a los datos de TEST para obtener las previsiones y poder posteriormente evaluar el modelo -----------")
predictions = model.transform(df_DT_test_assembled)

print("   ---------- Vemos resultados obtenidos -----------")
predictions.select("score_evaluation", "features", "rawPrediction", "probability", "prediction").show(100)


---------- Creamos el arbol de decision para la columna score_evaluation y entrenamos el modelo con los datos TRAIN -----------
 ---------- Aplicamos el modelo a los datos de TEST para obtener las previsiones y poder posteriormente evaluar el modelo -----------
 ---------- Vemos resultados obtenidos -----------
+----------------+--------------------+--------------------+--------------------+----------+
score_evaluation| features| rawPrediction| probability|prediction|
+----------------+--------------------+--------------------+--------------------+----------+
 2|[194.0,7.7,0.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,103.0,...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,20.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,2.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,0.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,10.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,38.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,17.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,0.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,0.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,0.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,7.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,264.0,...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,6.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,116.0,...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,18.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,0.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,0.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,15.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,45.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,15.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,8.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,21.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,0.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,0.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,12.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,0.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,13.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,73.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,13.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,8.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,0.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,0.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,0.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,55.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,23.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,46.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,85.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,4.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,40.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,11.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,176.0,...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,0.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,26.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,46.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,81.0,1...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,0.0,14...|[0.0,3769.0,28602...|[0.0,0.0130060147...| 2.0|
 2|[194.0,7.7,12.0,1...|[0.0,3769.0,28602...|[0.0,

### Ejercicio 11: Evaluar el modelo aplicándole un clasificador multiclase. Calcular la métrica 'accuracy', y conseguir el complementario para calcular el error. ###
* Evaluador: MulticlassClassificationEvaluator
  * Label: score_evaluation.
  * Prediction: prediction.
  * MetricName: accuracy.

In [39]:
# Escribir aquí el ejercicio
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="score_evaluation", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("   ---------- Vemos error obtenido -----------")
print("Error en el test = %g " % (1.0 - accuracy))


---------- Vemos error obtenido -----------
Error en el test = 0.0111917

## Spark ML: Pipelines ##

### Pipelines: Árboles de Decisión ###
Con el mismo concepto que con el KMeans, se va a diseñar el flujo para los árboles de decisión. Primero hay que aplicar los cambios de preprocesamiento vistos anteriormente al DataFrame inicial para preparalo.

### Ejercicio 12: Eliminar, de los dataframes df_spark_sql_train y df_spark_sql test, las variables 'Hotel_Address', 'Hotel_Name', 'Tags', 'Positive Review', 'Negative_Review' y 'score_string'. Llamarlos: df_DT_train y df_DT_test. ###

In [43]:
# Escribir aquí el ejercicio
# Hecho en un punto anterior
# df_DT_train = df_spark_sql_train.drop('Hotel_Address', 'Hotel_Name', 'Tags', 'Positive_Review', 'Negative_Review', 'score_string')
print("   ---------- Comprobamos que se han eliminado las columnas de df_DT_train -----------")
print(df_DT_train.printSchema())
print("   ---------- Comprobamos que se han eliminado las columnas de df_DT_test -----------")
print(df_DT_test.printSchema())

---------- Comprobamos que se han eliminado las columnas de df_DT_train -----------
root
-- Additional_Number_of_Scoring: integer (nullable = true)
-- Review_Date: string (nullable = true)
-- Average_Score: double (nullable = true)
-- Reviewer_Nationality: string (nullable = true)
-- Review_Total_Negative_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews: integer (nullable = true)
-- Review_Total_Positive_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
-- Reviewer_Score: double (nullable = true)
-- days_since_review: integer (nullable = true)
-- lat: float (nullable = true)
-- lng: float (nullable = true)
-- score_evaluation: integer (nullable = true)
-- Review_Date_index: double (nullable = false)
-- Reviewer_Nationality_index: double (nullable = false)

None
 ---------- Comprobamos que se han eliminado las columnas de df_DT_test -----------
root
-- Additional_Number_of_Scoring: integer (nullable = true)
-- Review_Date: string (nullable = true)
-- Average_Score: double (nullable = true)
-- Reviewer_Nationality: string (nullable = true)
-- Review_Total_Negative_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews: integer (nullable = true)
-- Review_Total_Positive_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
-- Reviewer_Score: double (nullable = true)
-- days_since_review: integer (nullable = true)
-- lat: float (nullable = true)
-- lng: float (nullable = true)
-- score_evaluation: integer (nullable = true)
-- Review_Date_index: double (nullable = false)
-- Reviewer_Nationality_index: double (nullable = false)

None

Después se diseña el flujo para este modelo, el cual será:

** StringIndexer --> VectorAssembler --> Decission Tree (Inicialización) --> Decission Tree (Entrenamiento) --> Modelo Decission Tree entrenado **

### Ejercicio 13: Recoger una lista con todos los StringIndexer a aplicar, y llamarla DT_string_indexers ###
 En lugar de sobreescribir cada vez el dataframe, crear una lista, y con el método 'append', se irán añadiendo todos los StringIndexers().

In [46]:
# Escribir aquí el ejercicio

print("   ---------- Definimos un indexer para cada columna tipo string -----------")
DT_string_indexers = []

for name, dtype in df_DT_train.dtypes:
  if dtype == "string":
    print("   ---------- Definimos indexer para la columna: ", name)
    DT_string_indexers.append(StringIndexer(inputCol=name, outputCol=name+"_index2"))
df_DT_train.printSchema() 

---------- Definimos un indexer para cada columna tipo string -----------
 ---------- Definimos indexer para la columna: Review_Date
 ---------- Definimos indexer para la columna: Reviewer_Nationality
root
-- Additional_Number_of_Scoring: integer (nullable = true)
-- Review_Date: string (nullable = true)
-- Average_Score: double (nullable = true)
-- Reviewer_Nationality: string (nullable = true)
-- Review_Total_Negative_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews: integer (nullable = true)
-- Review_Total_Positive_Word_Counts: integer (nullable = true)
-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
-- Reviewer_Score: double (nullable = true)
-- days_since_review: integer (nullable = true)
-- lat: float (nullable = true)
-- lng: float (nullable = true)
-- score_evaluation: integer (nullable = true)
-- Review_Date_index: double (nullable = false)
-- Reviewer_Nationality_index: double (nullable = false)

### Ejercicio 14: Guardar en la variable 'DT_vector_assembler' la aplicación del mismo VectorAssembler() del ejercicio 8. ###

In [48]:
# Escribir aquí el ejercicio
DT_vector_assembler = VectorAssembler(
    inputCols=["Additional_Number_of_Scoring", "Average_Score", "Review_Total_Negative_Word_Counts", "Total_Number_of_Reviews", "Review_Total_Positive_Word_Counts", "Total_Number_of_Reviews_Reviewer_Has_Given", "Reviewer_Score", "days_since_review", "Review_Date_index2", "Reviewer_Nationality_index2"],
    outputCol="features")

### Ejercicio 15: Crear una lista con el mombre de DT_pipeline_stages, y añadirle la lista de StringIndexers y el VectorAssembler (en este orden) ###

In [50]:
# Escribir aquí el ejercicio
DT_pipeline_stages = []
# Añadir la lista de String Indexers
DT_pipeline_stages = [str_indexer for str_indexer in DT_string_indexers]
DT_pipeline_stages.append(DT_vector_assembler)


### Ejercicio 16: Inicializar el modelo de árbol de decisión (mismas especificaciones que en el ej. 10), y añadirlo a la lista de pasos 'DT_pipeline_stages' ###

In [52]:
# Escribir aquí el ejercicio
print("   ---------- Creamos el arbol de decision para la columna score_evaluation y entrenamos el modelo con los datos TRAIN -----------")
dt_pipeline = DecisionTreeClassifier(labelCol="score_evaluation", featuresCol="features", maxBins=1000, maxDepth=1)
DT_pipeline_stages.append(dt_pipeline)


---------- Creamos el arbol de decision para la columna score_evaluation y entrenamos el modelo con los datos TRAIN -----------

### Ejercicio 17: Diseñar el Pipeline y aplicarlo sobre los datos de Train, llamándolo 'DT_pipeline_model' ###

In [54]:
# Escribir aquí el ejercicio

# Definimos el pipeline a partir de los pasos anteriores
pipeline_list = Pipeline(stages=DT_pipeline_stages)

# Aplicamos el pipeline sobre los datos TRAIN:
DT_pipeline_model = pipeline_list.fit(df_DT_train)


### Ejercicio 18: Aplicar el modelo resultante sobre los datos de test y evaluarlo al igual que se hizo en el ej. 11 ###

In [56]:
# Escribir aquí el ejercicio
DT_predictions = DT_pipeline_model.transform(df_DT_test)
DT_predictions.select("score_evaluation", "features", "rawPrediction", "probability", "prediction").show(100)

print(predictions.dtypes)
print(DT_predictions.dtypes)

print(type(predictions))
print(type(DT_predictions))

dtModel = DT_pipeline_model.stages[len(DT_pipeline_model.stages)-1]
evaluator2 = MulticlassClassificationEvaluator(
    labelCol="score_evaluation", predictionCol="prediction", metricName="accuracy")
# No entiendo que esta expresion de error...
accuracy2 = evaluator2.evaluate(DT_predictions)
print("   ---------- Vemos error obtenido -----------")
print("Error en el test = %g " % (1.0 - accuracy2))

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-4027673054087877> in <module> () 
 12 evaluator2 = MulticlassClassificationEvaluator(
 13 labelCol="score_evaluation", predictionCol="prediction", metricName="accuracy")
 ---> 14 accuracy2 = evaluator2 . evaluate ( DT_predictions ) 
 15 print ( " ---------- Vemos error obtenido -----------" ) 
 16 print ( "Error en el test = %g " % ( 1.0 - accuracy ) ) 

 /databricks/spark/python/pyspark/ml/evaluation.py in evaluate (self, dataset, params) 
 67 return self . copy ( params ) . _evaluate ( dataset ) 
 68 else : 
 ---> 69 return self . _evaluate ( dataset ) 
 70 else : 
 71 raise ValueError ( "Params must be a param map but got %s." % type ( params ) ) 

 /databricks/spark/python/pyspark/ml/evaluation.py in _evaluate (self, dataset) 
 97 """
 98 self . _transfer_params_to_java ( ) 
 ---> 99 return self . _java_obj . evaluate ( dataset . _jdf ) 
 100 
 101 def isLargerBetter ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1131 answer = self . gateway_client . send_command ( command ) 
 1132 return_value = get_return_value(
 -> 1133 answer, self.gateway_client, self.target_id, self.name)
 1134 
 1135 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.4-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 317 raise Py4JJavaError(
 318 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 319 format(target_id, ".", name), value)
 320 else : 
 321 raise Py4JError(

 Py4JJavaError : An error occurred while calling o1052.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 89.0 failed 1 times, most recent failure: Lost task 5.0 in stage 89.0 (TID 1329, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$5: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:423)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:191)
	at org.apache.spark.shuffle.sort.SortShuffleWriter.write(SortShuffleWriter.scala:63)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:96)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:53)
	at org.apache.spark.scheduler.Task.run(Task.scala:111)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:354)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: org.apache.spark.SparkException: Unseen label: Anguilla . To handle unseen labels, set Param handleInvalid to keep.
	at org.apache.spark.ml.feature.StringIndexerModel$$anonfun$5.apply(StringIndexer.scala:222)
	at org.apache.spark.ml.feature.StringIndexerModel$$anonfun$5.apply(StringIndexer.scala:208)
	... 16 more

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1683)
	at org.apache.spar